In [40]:
import numpy as np 
import scipy.optimize as sp
import datetime as dt 
import pandas as pd
from pandas_datareader import data as pdr 
import yfinance as yfin
import plotly.graph_objects as go
yfin.pdr_override()

In [2]:
def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks,start=start,end=end)
    stockData = stockData['Close']

    returns = stockData.pct_change() 
    meanReturns = returns.mean() 
    covMatrix = returns.cov()

    return meanReturns, covMatrix

def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252 # daily returns * trading days = yearly returns 
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) * 252)  # w'Sw, yearly standard deviation
    return returns, std 

def negativeSR(weights, meanReturns, covMatrix, rf = 0.05):
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - rf) / pStd 

def maxSR(meanReturns, covMatrix, rf = 0.05, constraintSet=(0,1)):
    """ Minimize Sharpe Ratio """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix, rf)
    constraints = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sp.minimize(negativeSR, numAssets*[1./numAssets], args = args, 
                         method = "SLSQP", bounds=bounds,constraints=constraints)
    return result 

In [32]:
stockList = ['NVDA','GOOG','META','SHOP', 'XOM','CRWD','ADBE','CM','BIDU']
endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

meanReturns, covMatrix = getData(stockList, startDate, endDate)

#weights = np.array([0.3, 0.3, 0.2, 0.2])
#returns, std = portfolioPerformance(weights, meanReturns, covMatrix)

result = maxSR(meanReturns, covMatrix)
print(result)

[*********************100%%**********************]  9 of 9 completed


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -3.0235109958826145
       x: [ 8.010e-16  3.151e-15  0.000e+00  2.574e-01  1.907e-02
            4.536e-02  3.990e-01  1.802e-15  2.792e-01]
     nit: 10
     jac: [ 1.219e+00  2.773e+00 -1.313e-01 -1.967e-01 -1.970e-01
           -1.969e-01 -1.967e-01  2.076e+00 -1.971e-01]
    nfev: 103
    njev: 10


In [33]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    """ Minimize Portfolio Variance """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sp.minimize(portfolioVariance, numAssets*[1./numAssets], args = args, 
                         method = "SLSQP", bounds=bounds,constraints=constraints)
    return result 

result = minimizeVariance(meanReturns, covMatrix)
print(result)


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.13101739410934934
       x: [ 8.535e-02  1.464e-02  2.870e-01  2.340e-02  1.299e-01
            3.181e-02  3.043e-02  2.960e-17  3.975e-01]
     nit: 6
     jac: [ 1.309e-01  1.310e-01  1.317e-01  1.311e-01  1.309e-01
            1.310e-01  1.311e-01  2.433e-01  1.306e-01]
    nfev: 60
    njev: 6


In [34]:
def portfolioReturn(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[0]
def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """ for each returnTarget, output weights such that min volatility """
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    
    constraints = ({"type": 'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget },
                    {"type": "eq", "fun": lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(constraintSet for asset in range(numAssets))
    effOpt = sp.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                         method = 'SLSQP', bounds = bounds, constraints = constraints)
    return effOpt 

In [51]:
def calculatedResults(meanReturns, covMatrix, rf = 0.05, constraintSet=(0,1)):
    """output max sr, min vol, efficient frontier"""
    # MAX SR PORTFOLIO 
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)
    
    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,2) for i in maxSR_allocation.allocation]

    # MIN VOL PORTFOLIO 
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)
    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,2) for i in minVol_allocation.allocation]

    # EFFICIENT FRONTIER 
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20)
    for target in targetReturns: 
        #print(target)
        #print(efficientOpt(meanReturns, covMatrix, target))
        #print("\n")
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun']) # only returns objective function value
 
    maxSR_returns, maxSR_std = round(maxSR_returns*100, 2), round(maxSR_std*100,2)
    minVol_returns, minVol_std = round(minVol_returns*100, 2), round(minVol_std*100, 2)

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns 

print(calculatedResults(meanReturns, covMatrix))

(81.8, 25.4,         allocation
Ticker            
ADBE          0.00
BIDU          0.00
CM            0.00
CRWD         25.74
GOOG          1.91
META          4.54
NVDA         39.90
SHOP          0.00
XOM          27.92, 20.77, 13.1,         allocation
Ticker            
ADBE          8.53
BIDU          1.46
CM           28.70
CRWD          2.34
GOOG         12.99
META          3.18
NVDA          3.04
SHOP          0.00
XOM          39.75, [0.1310164358215588, 0.13129595026402793, 0.13229279142340106, 0.13413765951403034, 0.13679631785370552, 0.14022254703883982, 0.1444754375885247, 0.1496333058102292, 0.15560659172476626, 0.162305265210584, 0.16964283677957662, 0.17754102145523773, 0.18592797983957549, 0.19474092838495943, 0.2039234615376026, 0.21342919935771876, 0.2232163949129091, 0.2332497320258267, 0.24350324219308372, 0.25400618610479103], array([0.20772958, 0.23984846, 0.27196734, 0.30408622, 0.3362051 ,
       0.36832397, 0.40044285, 0.43256173, 0.46468061, 0.49679949,
      

In [29]:
print(efficientOpt(meanReturns, covMatrix, 1)) # three optimization problems: min var, max sr, establish efficient frontier

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.3505418139313842
       x: [ 1.513e-01  1.749e-01  6.738e-01  0.000e+00]
     nit: 4
     jac: [ 1.400e-01  2.222e-01  4.311e-01  1.880e-01]
    nfev: 21
    njev: 4


In [52]:
def EF_graph(meanReturns, covMatrix, rf = 0.05, constraintSet = (0,1)):
    """ Plot min vol, max sr efficient frontier """
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, rf, constraintSet)

    # MAX SR 
    MaxSharpeRatio = go.Scatter(
        name = 'Maximum Sharpe Ratio',
        mode = 'markers',
        x = [maxSR_std],
        y = [maxSR_returns],
        marker = dict(color='red',size=14,line=dict(width=3,color='black'))
    )
    # MIN VOL 
    MinVol = go.Scatter(
        name = 'Minimum Volatility',
        mode = 'markers',
        x = [minVol_std],
        y = [minVol_returns],
        marker = dict(color='green',size=14,line=dict(width=3,color='black'))
    )
    # EFFICIENT FRONTIER 
    EF_curve = go.Scatter(
        name = 'Efficient Frontier',
        mode = 'lines',
        x = [round(ef_std*100,2) for ef_std in efficientList],
        y = [round(target*100,2) for target in targetReturns],
        line = dict(color='black', width = 4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimization with Efficient Frontier',
        yaxis = dict(title='Annualized Return (%)'),
        xaxis = dict(title="Annualized Volatility (%)"),
        showlegend = True ,
        legend = dict(
            x=0.75, y=0, traceorder = 'normal',
            bgcolor = '#E2E2E2',
            bordercolor = 'black',
            borderwidth = 2
        ),
        width = 800,
        height = 600,
    )

    fig = go.Figure(data=data, layout=layout)

    return fig.show() 

EF_graph(meanReturns, covMatrix)